In [2]:
import mysql.connector
import os
import pandas as pd


# HOST = 

# Connect to the database
connection = mysql.connector.connect(
    host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
    user="admin",
    passwd="cujwiq-suqhu2-bycpuB",
    database="web-app"
)

# Create a cursor
cursor = connection.cursor()

# Execute a query
cursor.execute("SELECT * FROM models")

# Fetch the results
results = cursor.fetchall()

# Print the results
for result in results:
    print(result)
    
# Close the connection
connection.close()

def execute_query(query):
    connection = mysql.connector.connect(
        host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
        user="admin",
        passwd="cujwiq-suqhu2-bycpuB",
        database="web-app"
    )
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    connection.close()
    return results
    


(1, 'hankam', 'model hankam', 1, datetime.datetime(2024, 6, 23, 2, 30, 59), datetime.datetime(2024, 6, 23, 2, 30, 59))


In [14]:
import pandas as pd
import json
import pyodbc
pyodbc.drivers()

['ODBC Driver 17 for SQL Server',
 'ODBC Driver 18 for SQL Server',
 'MySQL-NATUNA',
 'ODBC Drivers',
 'ODBC Connection Pooling']

# Read Model

In [15]:
file_name = 'SFD Hankam.mdl'
# file_name = 'Most Likely.mdl'

In [16]:
# Open the .mdl file in read mode
with open(file_name, 'r', encoding='utf-8') as file:
    # Read the content of the file
    mdl_content = file.read()

In [17]:
# Split the content of the file by the newline character
content_splitter = '********************************************************'

mdl_content = mdl_content.split(content_splitter)[0].replace('{UTF-8}', '\n').replace('\n','').replace('\t', '')
variables = mdl_content.split('|')
print(variables)

['Warships Send on Striking Mission=Chance of Striking*Number of Operating Warships~Ship/Month~', 'Number of Operating Warships= INTEG (Finished Striking+Warships Production Finished-Warships Send on Striking Mission,282)~Ship~', 'Chance of Striking=0.05~Dmnl/Month~0.05', 'Finished Striking=IF THEN ELSE( Number of Warships on Striking Mission>=0 , Number of Warships on Striking Mission\\/Duration Striking , 0 )~Ship/Month~', 'Number of Warships on Striking Mission= INTEG (Warships Send on Striking Mission-Finished Striking,0)~Ship~Warships Send on Striking Mission-Finished Striking', 'Duration Striking=0.25~Month~', 'Aircraft Strength=(Enablers Aircraft*0.5)+(Fighters Aircraft Score*0.5)~Percentage~', 'Begin of Warships Production=Warships Production~Ship/Month~', 'Combat Vehicle Increase=Combat Vehicle Purchase~Unit/Month~', 'Combat Vehicle Operating Duration=360~Month~', 'Combat Vehicle Purchase=50/12~Unit/Month~', 'Combat Vehicle Score=Number of Combat Vehicle/Optimal Combat Vehicle

In [18]:
len(variables)

48

In [19]:
result = []
for item in variables[:-1]:
    result.append(item.split('\t'))
result

[['Warships Send on Striking Mission=Chance of Striking*Number of Operating Warships~Ship/Month~'],
 ['Number of Operating Warships= INTEG (Finished Striking+Warships Production Finished-Warships Send on Striking Mission,282)~Ship~'],
 ['Chance of Striking=0.05~Dmnl/Month~0.05'],
 ['Finished Striking=IF THEN ELSE( Number of Warships on Striking Mission>=0 , Number of Warships on Striking Mission\\/Duration Striking , 0 )~Ship/Month~'],
 ['Number of Warships on Striking Mission= INTEG (Warships Send on Striking Mission-Finished Striking,0)~Ship~Warships Send on Striking Mission-Finished Striking'],
 ['Duration Striking=0.25~Month~'],
 ['Aircraft Strength=(Enablers Aircraft*0.5)+(Fighters Aircraft Score*0.5)~Percentage~'],
 ['Begin of Warships Production=Warships Production~Ship/Month~'],
 ['Combat Vehicle Increase=Combat Vehicle Purchase~Unit/Month~'],
 ['Combat Vehicle Operating Duration=360~Month~'],
 ['Combat Vehicle Purchase=50/12~Unit/Month~'],
 ['Combat Vehicle Score=Number of Com

In [20]:
print(result[4])
print(result[5])

['Number of Warships on Striking Mission= INTEG (Warships Send on Striking Mission-Finished Striking,0)~Ship~Warships Send on Striking Mission-Finished Striking']
['Duration Striking=0.25~Month~']


In [21]:
result[0][0]

'Warships Send on Striking Mission=Chance of Striking*Number of Operating Warships~Ship/Month~'

In [22]:
final = []
for item in result:
    item = item[0]
    temp = {}
    try:
        # print(item)
        temp_list = item.split('~')        
        # print(temp_list)
        
        # temp_list = temp_list[1].split('~')
        temp['name'] = temp_list[0].split('=')[0]
        # print(len(temp_list[0].split('=')[0]))
        temp['value'] = '='.join(temp_list[0].split('=')[1:]).strip()
        temp['level'] = temp_list[-2]
        temp['unit'] = temp_list[-1]
        
        # temp_list = temp_list[0].split('=')[0]
        
    except Exception as e:
        print('error')
        print(str(e))
        print(item)
        break
    final.append(temp)
    
final

[{'name': 'Warships Send on Striking Mission',
  'value': 'Chance of Striking*Number of Operating Warships',
  'level': 'Ship/Month',
  'unit': ''},
 {'name': 'Number of Operating Warships',
  'value': 'INTEG (Finished Striking+Warships Production Finished-Warships Send on Striking Mission,282)',
  'level': 'Ship',
  'unit': ''},
 {'name': 'Chance of Striking',
  'value': '0.05',
  'level': 'Dmnl/Month',
  'unit': '0.05'},
 {'name': 'Finished Striking',
  'value': 'IF THEN ELSE( Number of Warships on Striking Mission>=0 , Number of Warships on Striking Mission\\/Duration Striking , 0 )',
  'level': 'Ship/Month',
  'unit': ''},
 {'name': 'Number of Warships on Striking Mission',
  'value': 'INTEG (Warships Send on Striking Mission-Finished Striking,0)',
  'level': 'Ship',
  'unit': 'Warships Send on Striking Mission-Finished Striking'},
 {'name': 'Duration Striking', 'value': '0.25', 'level': 'Month', 'unit': ''},
 {'name': 'Aircraft Strength',
  'value': '(Enablers Aircraft*0.5)+(Fight

In [23]:
# new_result = []
# for item in result:
#     item_result = [row.replace('\n', '').replace('=', '') for row in item]
#     new_result.append(item_result)
# new_result
        

In [29]:
final_data = pd.read_json(json.dumps(final))
final_data

,name,value,level,unit
0,Warships Send on Striking Mission,Chance of Striking*Number of Operating Warships,Ship/Month,
1,Number of Operating Warships,INTEG (Finished Striking+Warships Production F...,Ship,
2,Chance of Striking,0.05,Dmnl/Month,0.05
3,Finished Striking,IF THEN ELSE( Number of Warships on Striking M...,Ship/Month,
4,Number of Warships on Striking Mission,INTEG (Warships Send on Striking Mission-Finis...,Ship,Warships Send on Striking Mission-Finished Str...
5,Duration Striking,0.25,Month,
6,Aircraft Strength,(Enablers Aircraft*0.5)+(Fighters Aircraft Sco...,Percentage,
7,Begin of Warships Production,Warships Production,Ship/Month,
8,Combat Vehicle Increase,Combat Vehicle Purchase,Unit/Month,
9,Combat Vehicle Operating Duration,360,Month,


In [30]:
final_data

,name,value,level,unit
0,Warships Send on Striking Mission,Chance of Striking*Number of Operating Warships,Ship/Month,
1,Number of Operating Warships,INTEG (Finished Striking+Warships Production F...,Ship,
2,Chance of Striking,0.05,Dmnl/Month,0.05
3,Finished Striking,IF THEN ELSE( Number of Warships on Striking M...,Ship/Month,
4,Number of Warships on Striking Mission,INTEG (Warships Send on Striking Mission-Finis...,Ship,Warships Send on Striking Mission-Finished Str...
5,Duration Striking,0.25,Month,
6,Aircraft Strength,(Enablers Aircraft*0.5)+(Fighters Aircraft Sco...,Percentage,
7,Begin of Warships Production,Warships Production,Ship/Month,
8,Combat Vehicle Increase,Combat Vehicle Purchase,Unit/Month,
9,Combat Vehicle Operating Duration,360,Month,


In [31]:
# insert final data to database
final_data = final_data.fillna('NULL')
final_data = final_data.replace('', 'NULL')
final_data

,name,value,level,unit
0,Warships Send on Striking Mission,Chance of Striking*Number of Operating Warships,Ship/Month,NULL
1,Number of Operating Warships,INTEG (Finished Striking+Warships Production F...,Ship,NULL
2,Chance of Striking,0.05,Dmnl/Month,0.05
3,Finished Striking,IF THEN ELSE( Number of Warships on Striking M...,Ship/Month,NULL
4,Number of Warships on Striking Mission,INTEG (Warships Send on Striking Mission-Finis...,Ship,Warships Send on Striking Mission-Finished Str...
5,Duration Striking,0.25,Month,NULL
6,Aircraft Strength,(Enablers Aircraft*0.5)+(Fighters Aircraft Sco...,Percentage,NULL
7,Begin of Warships Production,Warships Production,Ship/Month,NULL
8,Combat Vehicle Increase,Combat Vehicle Purchase,Unit/Month,NULL
9,Combat Vehicle Operating Duration,360,Month,NULL


In [33]:
final_data['model_id'] = 1
final_data

,name,value,level,unit,model_id
0,Warships Send on Striking Mission,Chance of Striking*Number of Operating Warships,Ship/Month,NULL,1
1,Number of Operating Warships,INTEG (Finished Striking+Warships Production F...,Ship,NULL,1
2,Chance of Striking,0.05,Dmnl/Month,0.05,1
3,Finished Striking,IF THEN ELSE( Number of Warships on Striking M...,Ship/Month,NULL,1
4,Number of Warships on Striking Mission,INTEG (Warships Send on Striking Mission-Finis...,Ship,Warships Send on Striking Mission-Finished Str...,1
5,Duration Striking,0.25,Month,NULL,1
6,Aircraft Strength,(Enablers Aircraft*0.5)+(Fighters Aircraft Sco...,Percentage,NULL,1
7,Begin of Warships Production,Warships Production,Ship/Month,NULL,1
8,Combat Vehicle Increase,Combat Vehicle Purchase,Unit/Month,NULL,1
9,Combat Vehicle Operating Duration,360,Month,NULL,1


In [34]:
query = f"INSERT INTO variables (name, value, level, unit,  model_id) VALUES {', '.join([str(tuple(row)) for row in final_data.values])}"
query

try:
    connection = mysql.connector.connect(
        host="db-natuna.ctmogcuxclxn.ap-southeast-1.rds.amazonaws.com",
        user="admin",
        passwd="cujwiq-suqhu2-bycpuB",
        database="web-app"
    )
    cursor = connection.cursor()
    cursor.execute(query)
    connection.commit()
    connection.close()
    print('success')
except Exception as e:
    print('error')
    print(str(e))
    connection.close()
    print('error')
    

success


In [39]:
# Open the .mdl file in read mode
with open(file_name, 'r', encoding='utf-8') as file:
    # Read the content of the file
    mdl_content = file.read()

In [41]:
# read view variables
mdl_content.split('VIEW-Naval Strength')[1].split('\n')


['',
 '$-1--1--1,0,|12||-1--1--1|-1--1--1|-1--1--1|-1--1--1|-1--1--1|96,96,121,0',
 '10,1,Number of Combat Vehicle,385,593,60,24,3,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,2,Marine Ability Score,248,185,75,30,8,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,3,Marine Forces Strength,385,293,58,58,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '1,4,2,3,1,0,0,0,0,192,0,-1--1--1,,1|(342,215)|',
 '10,5,Aircraft Strength,518,49,55,55,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '10,6,Warships Strength,1002,269,55,55,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',
 '10,7,Fighters Aircraft Score,348,-68,56,56,2,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,8,Enablers Aircraft,347,123,75,30,8,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '1,9,7,5,1,0,0,0,0,192,0,-1--1--1,,1|(428,-55)|',
 '1,10,8,5,1,0,0,0,0,192,0,-1--1--1,,1|(445,118)|',
 '10,11,Marine Personel Forces,-71,403,75,30,3,3,0,0,-1,0,0,0,0,0,0,0,0,0',
 '10,12,Harbour,736,362,49,49,2,3,0,20,-1,0,0,0,0-0-0,176-255-176,||B|0-0-0,0,0,0,0,0,0',

# Generate Model